In [1]:
%matplotlib inline

# Import Dependencies

In [2]:
# System & OS
import warnings
warnings.filterwarnings('ignore')

# Data Analysis
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from tqdm.notebook import tqdm

# Utility Functions
from utils import *

# Miscellaneous
from decimal import Decimal

# Mount Storage

To start, `cd` to where the data are stored.

In [6]:
%cd '../data/'

/Users/zach/Documents/Python/Protostellar-Luminosity/data


# Feature Engineering

Then, read in our previous results from `master_file.csv`.

In [7]:
df = pd.read_csv('master_file.csv', skiprows=1, names=['Model number',
                                                       'Class', 
                                                       'Mass ratio', 
                                                       'Timestep', 
                                                       'Inclination', 
                                                       'Wavelength (cm)', 
                                                       'Flux (erg cm^-2 s^-1)', 
                                                       'L_int (Lsun)'])
df

,Model number,Class,Mass ratio,Timestep,Inclination,Wavelength (cm),Flux (erg cm^-2 s^-1),L_int (Lsun)
0,1,0,0.034096,2,5,3.600030,1.171193e-18,0.001486
1,1,0,0.034096,2,5,2.400020,4.342741e-18,0.001486
2,1,0,0.034096,2,5,1.600013,1.734933e-17,0.001486
3,1,0,0.034096,2,5,1.300011,3.665945e-17,0.001486
4,1,0,0.034096,2,5,1.100009,6.839343e-17,0.001486
...,...,...,...,...,...,...,...,...
8993595,33,1,0.885142,379,85,0.000025,2.878049e-13,5.9644
8993596,33,1,0.885142,379,85,0.000020,1.755736e-14,5.9644
8993597,33,1,0.885142,379,85,0.000015,-1.782472e-16,5.9644
8993598,33,1,0.885142,379,85,0.000010,-1.308467e-17,5.9644


Next, perform the following feature modifications:

1. Drop the timestep and inclination columns.
2. Typecast mass ratio and luminosity columns to `float64`.
3. Convert wavelengths from cm to microns. $$\begin{align*}\lambda=\lambda_0\times10^4\end{align*}$$ where $\lambda_0$ denotes the initial wavelength (in cm), and $\lambda$ the corrected wavelength (in microns).
4. Further subdivide class 1 into 1a and 1b. $$\begin{align*}\text{class}=\begin{cases}0,&M<0.5\\1\text{a},&0.5\leq M<0.75\\1\text{b},&M\geq 0.75\end{cases}\end{align*}$$ where $M$ denotes the internal vs final mass ratio.
5. Compute the logged (`log10`) flux and internal luminosities. $$\begin{align*}\begin{split}F&=\log_{10}{\left(F_0\right)}\\L&=\log_{10}{\left(L_0\right)}\end{split}\end{align*}$$where $F_0$ and $F$ represent the flux values in linear and `log10` space, respectively (and equivalently for $L_0$ and $L$).
6. Drop rows with internal luminosity $L<0.1$.

In [8]:
# Drop irrelevant columns
df.drop(columns=['Timestep', 'Inclination'], inplace=True)

# Typecast objects to float64
df['Mass ratio'] = df['Mass ratio'].apply(lambda x: float(x))
df['L_int (Lsun)'] = df['L_int (Lsun)'].apply(lambda x: float(x))

# Convert wavelengths to microns
df['Wavelength (cm)'] = df['Wavelength (cm)'].apply(lambda x: x * 10000.0)
df.rename(columns={'Wavelength (cm)': 'Wavelength (microns)'}, inplace=True)

# Subdivide class 1 into 1a and 1b
df.loc[df['Mass ratio'] < 0.5, 'Class'] = '0'
df.loc[(df['Mass ratio'] >= 0.5) & (df['Mass ratio'] < 0.75), 'Class'] = '1a'
df.loc[df['Mass ratio'] >= 0.75, 'Class'] = '1b'

# Add columns with logged data
df['log(Flux)'] = np.log10(df['Flux (erg cm^-2 s^-1)'].values)
df['log(L_int)'] = np.log10(df['L_int (Lsun)'].values)

# Drop rows with L_int < 0.1
df = df[df['L_int (Lsun)'] >= 0.1]

# Print number of rows dropped from original dataframe
print(f'Number of rows dropped: {8993600 - len(df)} ({(8993600 - len(df)) / 8993600 * 100:.2f}%)')

Number of rows dropped: 1097100 (12.20%)


# Handle Missing Values

Since non-positive flux values are undefined in `log10` space, there are now rows with missing values (as a consequence of steps 6 & 7) that we must deal with accordingly.

In [9]:
# Replace Inf values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Print percentage of missing values
check_nan(df)

Percentage of missing values in each column:

log(Flux)                1.78
Model number             0.00
Class                    0.00
Mass ratio               0.00
Wavelength (microns)     0.00
Flux (erg cm^-2 s^-1)    0.00
L_int (Lsun)             0.00
log(L_int)               0.00
dtype: float64



Notice, there are relatively few missing values w.r.t. the full population. Hence, we opt to simply drop the rows containing `NaN` values.

In [10]:
# Drop missing values
df.dropna(axis=0, inplace=True)

# Print number of rows dropped from original dataframe
print(f'Number of rows dropped: {7896500 - len(df)} ({(7896500 - len(df)) / 8993600 * 100:.2f}%)')

Number of rows dropped: 140301 (1.56%)


# Data Analysis

Finally, we execute the following analysis steps:

1. Visualize flux as a function of luminosity in `log10` space.
2. Fit a linear regression model to the data.
3. Compute the associated linear coefficients (i.e. slope, intercept, and corresponding uncertainties).
4. Evaluate the goodness-of-fit (in both linear and `log10` space) via reduced-chi squared. $$\begin{align*}\chi_\nu^2=\frac{\chi^2}{\nu}\end{align*}$$ where $\chi$ represents a weighted sum of squared deviations, and $\nu$ the degree(s) of freedom. Further reading available [here](https://en.wikipedia.org/wiki/Reduced_chi-squared_statistic).
5. Calculate the correlation coefficients (in both linear and `log10` space) between flux and luminosity. $$\begin{align*}r=\frac{\sum_i(L_i-\bar{L})(F_i-\bar{F})}{\sqrt{\sum_i(L_i-\bar{L})^2\sum_i(F_i-\bar{F})^2}}\end{align*}$$ where $\bar{L}$ and $\bar{F}$ denote the mean luminosity and flux values, respectively. Further reading available [here](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient).

Note that these are repeated for each of the 100 wavelengths where data are available, which are further split along the 4 possible classes (0, 1a, 1b, All). An additional master plot containing data at all wavelengths was created as well.

In [8]:
# Write column headers for lin_coef.csv & corr_coef.csv
with open('lin_coef.csv', 'w') as coef, open('corr_coef.csv', 'w') as corr:
  coef.write('Class, Wavelength (microns), Slope, Unc_slope, Intercept, Unc_intercept\n')
  corr.write('Class, Wavelength (microns), Reduced chi-squared (linear space), Reduced chi-squared (log10 space), Correlation coefficient (linear space), Correlation coefficient (log10 space)\n')
coef.close()
corr.close()

In [9]:
# Initialize progress bar
with tqdm(total=df['Wavelength (microns)'].nunique(), leave=True) as pbar:
    # Partition data based on wavelength
    for wavelength in df['Wavelength (microns)'].unique():
        df_wavelength = df[df['Wavelength (microns)'].round(2) == round(wavelength, 2)]
        # For each wavelength, create a 4-figure subplot, where each subplot represents a unique class
        fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)
        fig.suptitle(f'Flux vs Internal Luminosity: {Decimal(wavelength):.2e} microns')
        # Customize axes
        ax[0][0].set_xscale('log')
        ax[0][0].set_yscale('log')
        ax[0][0].set_xlim([1e-1, 1e5])
        ax[0][0].set_ylim([1e-12, 1e-5])
        # Make plots
        sns.regplot(ax=ax[0, 0], data=df_wavelength[df_wavelength['Class'] == '0'], x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
        ax[0][0].set_title('Class 0')
        sns.regplot(ax=ax[1, 0], data=df_wavelength[df_wavelength['Class'] == '1a'], x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
        ax[1][0].set_title('Class 1a')
        sns.regplot(ax=ax[0, 1], data=df_wavelength[df_wavelength['Class'] == '1b'], x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
        ax[0][1].set_title('Class 1b')
        sns.regplot(ax=ax[1, 1], data=df_wavelength, x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
        ax[1][1].set_title('All classes')
        # Make legends & record metrics
        make_legend(df_wavelength, wavelength, ax)
        # Save figure
        plt.savefig(f'../data/Figures/flux_vs_lint_{Decimal(wavelength):.2e}.eps')
        # Clear figure
        plt.clf()
        # Update progress bar
        pbar.update(1)

  0%|          | 0/100 [00:00<?, ?it/s]

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

...

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

In [10]:
# Make master plot
fig, ax = plt.subplots(2, 2, figsize=(15, 15), sharex=True, sharey=True)
fig.suptitle(f'Flux vs Internal Luminosity: All wavelengths')
# Customize axes
ax[0][0].set_xscale('log')
ax[0][0].set_yscale('log')
ax[0][0].set_xlim([1e-1, 1e5])
ax[0][0].set_ylim([1e-12, 1e-5])
# Make plots
sns.regplot(ax=ax[0, 0], data=df[df['Class'] == '0'], x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
ax[0][0].set_title('Class 0')
sns.regplot(ax=ax[1, 0], data=df[df['Class'] == '1a'], x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
ax[1][0].set_title('Class 1a')
sns.regplot(ax=ax[0, 1], data=df[df['Class'] == '1b'], x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
ax[0][1].set_title('Class 1b')
sns.regplot(ax=ax[1, 1], data=df, x='L_int (Lsun)', y='Flux (erg cm^-2 s^-1)')
ax[1][1].set_title('All classes')
# Make legends & record metrics
make_legend(df, wavelength, ax)
# Save figure
plt.savefig(f'../data/Figures/flux_vs_lint_master.eps')
# Show figure
plt.show()

TypeError: make_legend() missing 1 required positional argument: 'ax'

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7ff514dda3b0> (for post_execute):


KeyboardInterrupt: 